In [1]:
import numpy as np
import numba
import params

In [2]:
species = [("preMiR", 1),
          ("preMiR_Dicer", 0),
          ("miRNA", 0),
          ("Dicer", 50)]

In [3]:
species

[('preMiR', 1), ('preMiR_Dicer', 0), ('miRNA', 0), ('Dicer', 50)]

In [45]:
init_dicer, init_premirna1, init_premirna2, init_premirna3, init_premirna4, \
init_premirna5, init_premirna1_dicer, init_premirna2_dicer, init_premirna3_dicer, \
init_premirna4_dicer, init_premirna5_dicer, init_mirna1, init_mirna2, init_mirna3, \
init_mirna4, init_mirna5 = params.init_values

In [9]:
ka1, ka_1, ka2, kb1, kb_1, kb2, kc1, kc_1, kc2, kd1, kd_1, kd2, ke1, ke_1, ke2 = params.theta

In [33]:
def generateSpecies(num_preMiR, pMiR, dicer_pMiR, miR):
    """
    Function to generate vector of initial concentrations for all species.
    
    Args:
    num_preMiR:    int, number of pre-miRNA species
    pMiR:          list of floats, initial concentration of pre-miRNA
    dicer_pMiR:    list of floats, initial concentration Dicer and pre-miRNA complexes
    miR:           list of floats, initial concentration of miRNA
    """
    
    assert num_preMiR == len(pMiR), "The number of concentrations given do not correspond with the number of species"
    assert num_preMiR == len(dicer_pMiR), "The number of concentrations given do not correspond with the number of species"
    assert num_preMiR == len(miR), "The number of concentrations given do not correspond with the number of species"
    
    init_concs = [pMiR[0], dicer_pMiR[0]]
    init_species = {"pMiR0": pMiR[0], "dicer_pMiR0": dicer_pMiR[0], "miR0": miR[0]}
    
    for i in range(1, num_preMiR):
        init_concs.append(pMiR[i])
        init_concs.append(dicer_pMiR[i])
        
        init_species["preMiR"+str(i)] = pMiR[i]
        init_species["dicer_preMiR"+str(i)] = dicer_pMiR[i]
        init_species["miR"+str(i)] = miR[i]
        
    return init_concs, init_species

In [86]:
def generateSystem(num_preMiR, k0, k1, k2):
    """
    Function to generate system of ODEs with n type miRNAs, all with customisable reaction rates.
    
    Args:
    num_preMiR:    int, number of pre-miRNA species
    k0:            list of floats, reaction rate for association of Dicer with pre-miRNA
    k1:            list of floats, reaction rate for disassociation of Dicer and pre-miRNA
    k2:            list of floats, reaction rate for miRNA formation
    """
    
    assert num_preMiR == len(k0), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    assert num_preMiR == len(k1), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    assert num_preMiR == len(k2), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    
    basic_system = [[k1[0], -k0[0]], #dWT/dt start with dicer_pre-miR, second term dicer x pre-miR
              [-(k1[0] + k2[0]), k0[0]], #dDicerWT/dt
              [k2[0], 0]] #dmiRNA/dt
    dicer = [(k1[0] + k2[0]), -k0[0]] #last index:  dDicer/dt
    
    for i in range(1, num_preMiR):
        
        basic_system0 = [[k1[i], -k0[i]], #dWT/dt start with dicer_pre-miR, second term dicer x pre-miR
              [-(k1[i] + k2[i]), k0[0]], #dDicerWT/dt
              [k2[i], 0]] #dmiRNA/dt
        dDicer = [(k1[i] + k2[i]), -k0[i]] #last index:  dDicer/dt
        
        for j in range(3):
            #add empty rows at bottom of matrix
            basic_system.append([0]*len(basic_system[0]))
        for k in range(len(basic_system)):
            if k < len(basic_system) - 3:
                #add 0s to end of each row unless last 3
                basic_system[k] = basic_system[k] + [0]*2 #to keep it flat
            else:
                #add correct equation to end of row
                if k == len(basic_system) - 3:
                    basic_system[k] = basic_system[k] + basic_system0[0][:2]             
                elif k == len(basic_system) - 2:
                    basic_system[k] = basic_system[k] + basic_system0[1][:2]
                elif k == len(basic_system) - 1:
                    basic_system[k] = basic_system[k] + basic_system0[2][:2]
        #update dicer
        dicer = dicer + dDicer
        
    return basic_system + [dicer]

In [46]:
test = generateSpecies(5, [init_premirna1]*5, [init_dicer*init_premirna1]*5, \
                       [init_mirna1]*5)

In [28]:
timeit(generateSpecies(5, [1]*5, [0]*5, [0]*5))

4.97 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [139]:
test[0]

[1, 50, 1, 50, 1, 50, 1, 50, 1, 50]

In [87]:
test2 = generateSystem(5, [ka1]*5, [ka_1]*5, [ka2]*5)

In [89]:
len(test2)

16

In [42]:
timeit(generateSystem(5, [ka1]*5, [ka_1]*5, [ka2]*5))

22.7 µs ± 675 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [137]:
def test_model(t, init_values, ka, kb, kc):
    """
    Function to pass to generate, pass ODEs to ODE solver.
    
    Args:
    init_values:    Array of initial concentration of each initial concentration of every species, min 4
    ka:             Array of reaction rates for formation of pre-miR - dicer commplex
    kb:             Array of reaction rates for dissociation of pre-miR - dicer complex
    kc:             Array of reaction rates for catalysis of pre-miR to miR by dicer
    """
    
    concs_len = len(init_values)
    
    assert int(concs_len/2) == len(ka), "The number of reactions do not agree with the number of reaction rates given."
    assert int(concs_len/2) == len(kb), "The number of reactions do not agree with the number of reaction rates given."
    assert int(concs_len/2) == len(kc), "The number of reactions do not agree with the number of reaction rates given."
    
    ODEs = generateSystem(int(concs_len/2), ka, kb, kc)
    #output = [[0]]*len(ODEs)
    output = []
    
    for i in range(len(ODEs)):
        #utput[i] = output[i][0] + [(sum(ODEs[i] * init_values))]
        #utput.append(sum(ODEs[i] * init_values))
        #rint(output[i])
        if i == int(len(ODEs) - 1):
            exec('dicer = sum(ODEs[i] * init_values)')
            exec('output.append(dicer)')
            exec('print(i, dicer)')
        elif (i + 1) % 3 == 0:
            exec('mirna' + str(i) + ' = sum(ODEs[i] * init_values)')
            exec('output.append(mirna' + str(i) + ')')
            exec('print(i, mirna' + str(i) + ')')
        elif (i + 1) % 2 == 0:
            exec('pmiR_dicer' + str(i) + ' = sum(ODEs[i] * init_values)')
            exec('output.append(pmiR_dicer' + str(i) + ')')
            exec('print(i, pmiR_dicer' + str(i) + ')')
        else:
            exec('pmiR' + str(i) + ' = sum(ODEs[i] * init_values)')
            exec('output.append(pmiR' + str(i) + ')')         
            exec('print(i, pmiR' + str(i) + ')')
                 
    return output[:]

In [17]:
from scipy.integrate import solve_ivp

In [62]:
init_conc = test[0]

In [138]:
res1 = solve_ivp(test_model, (0, int(params.minutes)), init_conc, \
                 args=([ka1]*5, [ka_1]*5, [ka2]*5))

0 -25.30294992
1 25.12263892
2 0.180311
3 -25.30294992
4 25.12263892
5 0.180311
6 -25.30294992
7 25.12263892
8 0.180311
9 -25.30294992
10 25.12263892
11 0.180311
12 -25.30294992
13 25.12263892
14 0.180311
15 -125.6131946


ValueError: operands could not be broadcast together with shapes (16,) (10,) 

In [160]:
def dynamic_model(t, init_values, ka, kb, kc):
    """
    Dynamic model for ODE system creation and solving
    
    init_values:    List of initial concentration of free pre-miRNA concentration for
                    each pre-mirna, pre-mirna x dicer concentration and dicer 
                    concentration. Alternating pre-mirna (a) and pre-mirna dicer (b),
                    with final dicer concentration (c) i.e. [a, b, a, b, a, b, c]
    ka:             Array of reaction rates for formation of pre-miR - dicer commplex
    kb:             Array of reaction rates for dissociation of pre-miR - dicer complex
    kc:             Array of reaction rates for catalysis of pre-miR to miR by dicer
    """
    
    assert len(init_values-1) % 2 == 0, "You have supplied an uneven number of \
    pre-mirna, dicer x pre-mirna concentrations"
    assert int((len(init_values) - 1)/2) == len(ka), "The number of reactions do not \
    agree with the number of reaction rates given."
    assert int((len(init_values) - 1)/2) == len(kb), "The number of reactions do not \
    agree with the number of reaction rates given."
    assert int((len(init_values) - 1)/2) == len(kc), "The number of reactions do not \
    agree with the number of reaction rates given."
    
    #calculate number of miRNAs
    n_mirna = int(init_values/2)
    
    output = []
    dicer_strings = ""
    
    for i in range(n_mirna):
        exec('premirna' + str(i) + ' = init_values[i+1] * kb[i] - \
        init_values[i] * init_values[-1] * ka[i]')
        exec('output.append(premirna' + str(i) + ')')
        
        exec('pmiR_dicer' + str(i) + ' = init_values[i] * init_values[-1] * ka[i] - \
        init_values[i+1] * kb[i]')
        exec('output.append(pmiR_dicer' + str(i) + ')')
        
        exec('mirna' + str(i) + ' = init_values[i+1] * kc[i]')
        exec('output.append(mirna' + str(i) + ')')
    
        if i == 0:
            dicer_strings = dicer_strings + 'init_values[i+1] * (kb[i] + kc[i]) - \
            init_values[-1] * init_values[i] * ka[i]'
        
        else:
            dicer_strings = dicer_strings + '+ init_values[i+1] * (kb[i] + kc[i]) - \
            init_values[-1] * init_values[i] * ka[i]'
    
    exec('dicer = ' + dicer_strings)
    exec('output.append(dicer)')
    
    return output
    

In [146]:
def make_inits(num_preMiR, pMiR, dicer_pMiR, dicer):
    """
    Function to generate vector of initial concentrations for all species.
    
    Args:
    num_preMiR:    int, number of pre-miRNA species
    pMiR:          list of floats, initial concentration of pre-miRNA
    dicer_pMiR:    list of floats, initial concentration Dicer and pre-miRNA complexes
    dicer:         float, initial concentration of dicer
    """
    
    assert num_preMiR == len(pMiR), "The number of concentrations given do \
    not correspond with the number of species"
    assert num_preMiR == len(dicer_pMiR), "The number of concentrations given do not \
    correspond with the number of species"
    
    init_concs = [pMiR[0], dicer_pMiR[0]]
    init_species = {"pMiR0": pMiR[0], "dicer_pMiR0": dicer_pMiR[0]}
    
    for i in range(1, num_preMiR):
        init_concs.append(pMiR[i])
        init_concs.append(dicer_pMiR[i])
        
        init_species["preMiR"+str(i)] = pMiR[i]
        init_species["dicer_preMiR"+str(i)] = dicer_pMiR[i]
    
    init_concs.append(dicer)
    init_species["dicer"] = dicer
        
    return init_concs, init_species

In [147]:
inits = make_inits(5, [params.init_premirna1] * 5, [params.init_premirna1_dicer] * 5, \
                   params.init_dicer)

In [148]:
inits

([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 50],
 {'pMiR0': 1,
  'dicer_pMiR0': 0,
  'preMiR1': 1,
  'dicer_preMiR1': 0,
  'preMiR2': 1,
  'dicer_preMiR2': 0,
  'preMiR3': 1,
  'dicer_preMiR3': 0,
  'preMiR4': 1,
  'dicer_preMiR4': 0,
  'dicer': 50})

In [159]:
res2 = solve_ivp(dynamic_model, (0, int(params.minutes)), inits, \
                 args=([ka1]*5, [ka_1]*5, [ka2]*5))

ValueError: setting an array element with a sequence.